# Set up your environment

Let's use a `.env` file and the `dotenv` library. This allows you to keep your API key secure and separated from your code.

Here’s how you can setup to load the OpenAI API key using a `.env` file:

### Step 1: Install Dependencies
Make sure you have `python-dotenv` installed if you're using `.env` files:
```bash
pip install python-dotenv
pip install -U langsmith
pip install openai
```

### Step 2: Create a `.env` File
In your project directory, create a `.env` file with the following content:
```
OPENAI_API_KEY=your-api-key-here
```

### Step 3: Test

In [4]:
import os
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
langchain_api_key = get_env_var("LANGCHAIN_API_KEY")
langchain_tracing_v2 = get_env_var("LANGCHAIN_TRACING_V2")
openai_api_key = get_env_var("OPENAI_API_KEY")
tavily_api_key = get_env_var("TAVILY_API_KEY")

# Now, you can use the keys in our setup as needed
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_api_key)

In [5]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_e375328146', 'finish_reason': 'stop', 'logprobs': None}, id='run-16722a96-9796-468a-8b1e-8e388edba9b5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

In [6]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3537616b13', 'finish_reason': 'stop', 'logprobs': None}, id='run-a7c19b00-e391-46d7-a459-07b9ff02812d-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [7]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-90a6694a-76b6-4781-b6f7-36c0e786d2d1-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [8]:
# Set up Tavily using the API key from the environment
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3, api_key=tavily_api_key)

In [10]:
# Perform a search with Tavily
search_docs = tavily_search.invoke("What is LangGraph?")
search_docs

[{'url': 'https://www.langchain.com/langgraph',
  'content': 'LangGraph is a stateful, orchestration framework that brings added control to agent workflows. LangGraph Cloud is a service for deploying and scaling LangGraph applications, with a built-in Studio for prototyping, debugging, and sharing LangGraph applications.'},
 {'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner. It simplifies the development process by enabling the creation of cyclical graphs, which are essential for developing ...'},
 {'url': 'https://towardsdatascience.com/from-basics-to-advanced-exploring-langgraph-e8c1cf4db787',
  'content': "LangGraph is a low-level framework that offers extensive customisation options, allowing you to build precisely what you need. S

In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
openai_api_key = get_env_var("OPENAI_API_KEY")

# Initialize the OpenAI LLM via LangChain
gpt_chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

# Define a simple prompt template for conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a helpful assistant. User asks: {user_input}. How would you respond?"
)

# Instead of using RunnableSequence, directly chain the prompt and model using the | operator
conversation_chain = prompt_template | gpt_chat

try:
    # Start callback for tracing OpenAI usage
    with get_openai_callback() as cb:
        print("LangChain tracing started successfully.")

        # Example conversation - LangChain will trace this operation
        user_input = "What are the benefits of using transformers in natural language processing?"
        response = conversation_chain.invoke({"user_input": user_input})

        print("Agent's Response:")
        print(response)

        # You can extend this conversation further
        follow_up = "Can you explain the difference between GPT and BERT?"
        follow_up_response = conversation_chain.invoke({"user_input": follow_up})

        print("Follow-up Response:")
        print(follow_up_response)

        # Output callback information
        print(f"\nTotal Tokens Used: {cb.total_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    print("LangChain tracing ended successfully.")

except Exception as e:
    print(f"Error with LangChain tracing: {e}")


LangChain tracing started successfully.
Agent's Response:
content='Transformers have revolutionized natural language processing (NLP) by significantly improving the performance of various NLP tasks. Some of the key benefits of using transformers in NLP include:\n\n1. Enhanced performance: Transformers have shown superior performance compared to traditional NLP models, especially in tasks like language translation, text generation, and sentiment analysis.\n\n2. Attention mechanism: Transformers use an attention mechanism that allows the model to focus on different parts of the input sequence, enabling better understanding and capturing of long-range dependencies in the text.\n\n3. Scalability: Transformers are highly scalable and can be easily adapted to different languages and tasks without the need for extensive retraining.\n\n4. Transfer learning: Transformers support transfer learning, allowing pre-trained models to be fine-tuned on specific tasks with minimal data, leading to faste